In [4]:
import pandas as pd 
import os
import Bio
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import ClustalwCommandline
import collections
from Bio import AlignIO
import random
import numpy as np
import sys

#sys.path.append(os.path.abspath('/home/nimar/PLP_directRNA_design_V2/PLP_directRNA_design/'))
#from PLP_directRNA_design import probedesign as plp

In [ ]:
number_of_seqs= 10 # number of regions mapped/PLP
final_designed=5 # number of PLPs designed at the end/gene
plp_length=30
mismatches = 6


In [21]:
# Define IUPAC nucleotide codes for mismatches
IUPAC_CODES = {
    "R": ["A", "G"],
    "Y": ["C", "T"],
    "S": ["G", "C"],
    "W": ["A", "T"],
    "K": ["G", "T"],
    "M": ["A", "C"],
    "B": ["C", "G", "T"],
    "D": ["A", "G", "T"],
    "H": ["A", "C", "T"],
    "V": ["A", "C", "G"],
    "N": ["A", "C", "G", "T"]  # Any base
}

ligation_junctions_dict = {
    'TA': 'preferred', 'GA': 'preferred', 'AG': 'preferred',
    'TT': 'neutral', 'CT': 'neutral', 'CA': 'neutral',
    'TC': 'neutral', 'AC': 'neutral', 'CC': 'neutral',
    'TG': 'neutral', 'AA': 'neutral',
    'CG': 'non-preferred', 'GT': 'non-preferred',
    'GG': 'non-preferred', 'GC': 'non-preferred'
}

def evaluate_ligation_junction(targets, iupac_mismatches=None, plp_length=30):
    """
    Evaluates the ligation junction of a probe and introduces mismatches if needed.

    Args:
        probe_seq (str): The probe sequence.
        iupac_mismatches (list of tuples): List of positions and IUPAC codes to introduce mismatches.
                                            Example: [(5, 'R'), (10, 'Y')]
        plp_length (int): Probe length (default: 30).

    Returns:
        tuple: (updated probe sequence, ligation junction category)
    """
    # Extract the ligation junction (2 bases around the center of the probe)
    junction_position = int((plp_length / 2) - 1)
    for idx in targets.index:
        #print(f"Index: {idx}, Row Data: {targets.loc[idx]}")
        probe_seq = targets.loc[idx]['Sequence']
        ligation_junction = probe_seq[junction_position] + probe_seq[junction_position + 2]

        # Determine the ligation status
        ligation_status = ligation_junctions_dict.get(ligation_junction, "non-preferred")
        targets.loc[idx]['ligation_status'] = ligation_status

        # Apply IUPAC mismatches if provided
        num_mismatches = len(iupac_mismatches)
        if iupac_mismatches:
            for r in range(1, num_mismatches + 1):
                for subset in itertools.combinations(range(num_mismatches), r):
                    selected_mismatches = [iupac_mismatches[i] for i in subset]
                    replacement_options = [IUPAC_CODES[symbol] for pos, symbol in selected_mismatches]
                    # Generate all possible combinations of replacements
                    for replacement in itertools.product(*replacement_options):
                        new_probe_seq = list(probe_seq)
                        new_id_suffix = []
                        for (pos, iupac_symbol), new_base in zip(selected_mismatches, replacement):
                            new_probe_seq[pos] = new_base
                            new_id_suffix.append(f"{pos}_{iupac_symbol}_{new_base}")
                        new_probe_seq = "".join(new_probe_seq)
                        new_probe_id = f"{idx}|{'_'.join(new_id_suffix)}"
                        # Revaluate the ligation junction
                        new_ligation_junction = new_probe_seq[junction_position] + new_probe_seq[junction_position + 2]
                        new_ligation_status = ligation_junctions_dict.get(new_ligation_junction, "non-preferred")
                        new_row = targets.loc[idx].copy()
                        new_row['Sequence'] = new_probe_seq
                        new_row['Ligation junction'] = new_ligation_status
                        targets.loc[new_probe_id] = new_row
            
    
    return targets


In [60]:
prob_seq = 'AGAGGAGGAGAGCAAAGAGGCCAGTGCTCT'
iupac_mismatches = [(5, 'R'), (10, 'Y')]
plp_length = 30
probeSeq, ligation_status = evaluate_ligation_junction(prob_seq, iupac_mismatches, plp_length)
print(probeSeq, ligation_status)


TypeError: 'builtin_function_or_method' object is not iterable

In [61]:
import os
import pandas as pd
junction_position = int((plp_length / 2) - 1)

targets = pd.read_csv('../targets.txt', sep='\t', index_col=0)
for idx in targets.index:
    #print(f"Index: {idx}, Row Data: {targets.loc[idx]}")
    probe_seq = targets.loc[idx]['Sequence']
    ligation_junction = probe_seq[junction_position] + probe_seq[junction_position + 2]

    # Determine the ligation status
    ligation_status = ligation_junctions_dict.get(ligation_junction, "non-preferred")
    targets.loc[idx]['ligation_status'] = ligation_status
    #print(f"Ligation junction: {ligation_junction} ({ligation_status})")
    # Apply IUPAC mismatches if provided
    if iupac_mismatches:
        probe_seq = list(probe_seq)  # Convert to list for mutability
        for pos, iupac_symbol in iupac_mismatches:
            if 0 <= pos < len(probe_seq) and iupac_symbol in IUPAC_CODES:
                for base in IUPAC_CODES[iupac_symbol]:
                    new_probe_seq = probe_seq.copy()
                    new_probe_seq[pos] = base
                    new_probe_seq = "".join(new_probe_seq)  # Convert back to string
                    new_probe_id = f"{idx}|{pos}_{iupac_symbol}_{base}"
                    new_row = targets.loc[idx].copy()
                    new_row['Sequence'] = new_probe_seq
                    new_row['Ligation junction'] = ligation_junctions_dict.get(
                        new_probe_seq[junction_position] + new_probe_seq[junction_position + 2], "non-preferred"
                    )
                    targets.loc[new_probe_id] = new_row


targets

/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_81765/2685275217.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets.loc[idx]['ligation_status'] = ligation_status
/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_81765/2685275217.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets.loc[idx]['ligation_status'] = ligation_status
/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_81765/2685275217.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,Gene,Region,Sequence,GC,Coverage,Ligation junction
Probe_id,,,,,,
Prlh|90880853-90880882,Prlh,1:90880853-90880882,GTGCTTGCTGCTGCTAGGCTTAGTCCTCCC,60.000000,1.0,neutral
Prlh|90880854-90880883,Prlh,1:90880854-90880883,TGCTTGCTGCTGCTAGGCTTAGTCCTCCCA,56.666667,1.0,preferred
Prlh|90880855-90880884,Prlh,1:90880855-90880884,GCTTGCTGCTGCTAGGCTTAGTCCTCCCAG,60.000000,1.0,non-preferred
Prlh|90880856-90880885,Prlh,1:90880856-90880885,CTTGCTGCTGCTAGGCTTAGTCCTCCCAGG,60.000000,1.0,non-preferred
Prlh|90880857-90880886,Prlh,1:90880857-90880886,TTGCTGCTGCTAGGCTTAGTCCTCCCAGGA,56.666667,1.0,neutral
...,...,...,...,...,...,...
Grik2|49659291-49659320|10_Y_T,Grik2,10:49659291-49659320,TAAAGTCCTGTTCTGCTTGTTGTGGATCGG,50.000000,6.0,non-preferred
Grik2|49659292-49659321|5_R_A,Grik2,10:49659292-49659321,AAAGTACTGCTCTGCTTGTTGTGGATCGGA,50.000000,6.0,neutral
Grik2|49659292-49659321|5_R_G,Grik2,10:49659292-49659321,AAAGTGCTGCTCTGCTTGTTGTGGATCGGA,50.000000,6.0,neutral


In [59]:
import itertools

num_mismatches = len(iupac_mismatches)
for subset in itertools.combinations(range(num_mismatches), 2):
    selected_mismatches = [iupac_mismatches[i] for i in subset]
    replacement_options = [IUPAC_CODES[symbol] for pos, symbol in selected_mismatches]
    for replacement in itertools.product(*replacement_options):
        print(replacement)
    #print(replacement_options)
    #for pos, symbol in selected_mismatches:
        #print(pos, symbol)
        #print(IUPAC_CODES[symbol])



('A', 'C')
('A', 'T')
('G', 'C')
('G', 'T')


In [10]:
import pandas as pd 
import os
cur_path = os.chdir('/Users/nimra236/Dropbox/NBIS/Projects/PLP_directRNA_design_V2/')
print(cur_path)

selected_features = 'extract_features_output.txt'
selected_features= pd.read_csv(selected_features, sep='\t')
print(selected_features.columns)

None
Index(['seqname', 'start', 'end', 'strand', 'gene_name', 'transcript_id',
       'coverage', 'region'],
      dtype='object')


# Extracting sequence from gtf/fasta

In [26]:
def extract_transcripts_biopython(genome_fasta, annotation_gtf, output_fasta):
    """
    Extracts coding sequences (CDS) of transcripts using Biopython.
    """
    start_time = time.time()
    
    # Load genome sequences
    genome = SeqIO.to_dict(SeqIO.parse(genome_fasta, "fasta"))
    
    # Create GTF database
    db = gffutils.create_db(annotation_gtf, dbfn='annotation.db', force=True, keep_order=True, 
                            merge_strategy="merge", disable_infer_genes=True)
    
    transcripts = defaultdict(str)
    
    # Extract CDS sequences for each transcript
    for transcript in db.features_of_type("transcript"):
        chrom = transcript.seqid
        cds_seq = ""
        
        for cds in db.children(transcript, featuretype="CDS", order_by="start"):
            start = cds.start - 1  # Convert to 0-based index
            end = cds.end
            
            if chrom in genome:
                seq_fragment = genome[chrom].seq[start:end]
                if cds.strand == "-":
                    seq_fragment = seq_fragment.reverse_complement()
                cds_seq += str(seq_fragment)
        
        if cds_seq:
            transcripts[transcript.id] = cds_seq
    
    # Write output
    with open(output_fasta, "w") as f:
        for tid, seq in transcripts.items():
            f.write(f">{tid}\n{seq}\n")
    
    elapsed_time = time.time() - start_time
    print(f"Biopython CDS extraction completed in {elapsed_time:.2f} seconds. Output saved to: {output_fasta}")


In [23]:
extract_transcripts_biopython('../data/Mus.fa', annotation_gtf='../data/tmp.gtf', output_fasta='tmp.fasta')

Biopython CDS extraction completed in 4.20 seconds. Output saved to: tmp.fasta


# Find targets

In [ ]:
import sys
import os
import pandas as pd
import dnaio
from Bio import SeqIO
from Bio.Seq import Seq
from cutadapt.adapters import BackAdapter
from dataclasses import dataclass
from io import StringIO
import csv

import pandas as pd 
from pandas import DataFrame
import Bio
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import collections
from Bio import AlignIO
import random
import numpy as np
import argparse
import re
import os 
import multiprocessing
import subprocess
import itertools


import tqdm

# Dictionaries

## IUPAC dictionary
IUPAC_CODES = {
    "R": ["A", "G"],
    "Y": ["C", "T"],
    "S": ["G", "C"],
    "W": ["A", "T"],
    "K": ["G", "T"],
    "M": ["A", "C"],
    "B": ["C", "G", "T"],
    "D": ["A", "G", "T"],
    "H": ["A", "C", "T"],
    "V": ["A", "C", "G"],
    "N": ["A", "C", "G", "T"],  
    "A": ["A"],
    "C": ["C"],
    "G": ["G"],
    "T": ["T"]
}

## Ligation junction dictionary
ligation_junctions_dict = {'TA': 'preferred',
                        'TA': 'preferred',
                        'GA': 'preferred',
                        'AG': 'preferred',
                        'TT': 'neutral',
                        'CT': 'neutral',
                        'CA': 'neutral',
                        'TC': 'neutral',
                        'AC': 'neutral',
                        'CC': 'neutral',
                        'TG': 'neutral',
                        'AA': 'neutral', 
                        'CG': 'non-preferred', 
                        'GT': 'non-preferred',
                        'GG': 'non-preferred',
                        'GC': 'non-preferred'}


@dataclass
class Alignment:
    query_name: str
    target_name: str
    target_start: int
    target_end: int
    mismatches: int
    query_sequence: str
    target_sequence: str



def find_probes_in_targets(targets_df, reference_fasta, max_errors=1, output_file=None):
    """
    Function to check specificity of extracted probes against a reference genome.
    Uses Cutadapt's aligner for finding target regions.

    Args:
        targets_df (DataFrame): DataFrame containing extracted probe sequences.
        reference_fasta (str): Path to the reference genome FASTA.
        max_errors (int): Maximum number of allowed mismatches.
        output_file (str, optional): Path to save the results as a CSV file.

    Returns:
        DataFrame: A DataFrame containing matched probe alignments.
    """
    results = []

    with dnaio.open(reference_fasta) as references:
        references = list(references)  # Load references into a list
        total_probes = len(targets_df)  # Get total probe count

        with tqdm(total=total_probes, desc="Testing probes for specificity", unit=" alignments") as pbar:
            for reference_record in references:
                ref_id = reference_record.id
                ref_seq = reference_record.sequence

                for _, row in targets_df.iterrows():
                    probe_id = row["Probe_id"]
                    probe_seq = row["Sequence"]

                    adapter = BackAdapter(probe_seq, max_errors=max_errors, min_overlap=len(probe_seq), indels=False)
                    aligner = adapter.aligner

                    # Forward strand search
                    for t_start, t_end, errors, target_seq in find_all(ref_seq, aligner):
                        results.append(Alignment(
                            query_name=probe_id,
                            target_name=ref_id,
                            target_start=t_start + 1,  # Convert to 1-based index
                            target_end=t_end,
                            mismatches=errors,
                            query_sequence=probe_seq,
                            target_sequence=target_seq
                        ))
                    pbar.update(1)  # Update progress bar

                    # Reverse complement search
                    rev_ref_seq = str(Seq(ref_seq).reverse_complement())
                    adapter = BackAdapter(probe_seq, max_errors=max_errors, min_overlap=len(probe_seq), indels=False)
                    aligner = adapter.aligner

                    for t_start, t_end, errors, target_seq in find_all(rev_ref_seq, aligner):
                        results.append(Alignment(
                            query_name=probe_id,
                            target_name=f"{ref_id}(reverse)",
                            target_start=t_start + 1,
                            target_end=t_end,
                            mismatches=errors,
                            query_sequence=probe_seq,
                            target_sequence=target_seq
                        ))
                    pbar.update(1)  # Update progress bar again for reverse search

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Save results to file if requested
    if output_file:
        results_df.to_csv(output_file, index=False)
        sys.stderr.write(f"Results saved to {output_file}\n")

    return results_df


def find_all(ref, aligner):
    """Find all occurrences of a probe in a reference sequence."""
    offset = 0
    while True:
        result = aligner.locate(ref)
        if result is None:
            break
        ref_start, ref_end, query_start, query_end, score, errors = result
        t_start = query_start + offset
        t_end = query_end + offset
        target_seq = ref[query_start:query_end]
        yield (t_start, t_end, errors, target_seq)
        offset += query_start + 1
        ref = ref[query_start + 1:]


def find_targets(selected_features, fasta_file, reference_fasta, plp_length=30, min_coverage=1,
                 output_file='Candidate_probes', gc_min=50, gc_max=65, num_probes=10,
                 iupac_mismatches=None, max_errors=1, check_specificity=False):
    """
    Extract target sequences based on defined probe criteria and optionally check specificity.

    Args:
        selected_features (str): Path to the selected features file (TSV format).
        fasta_file (str): Path to the indexed FASTA file.
        output_file (str): Path to the output file.
        plp_length (int): Probe length (default: 30).
        min_coverage (int): Minimum coverage of the region.
        gc_min (int): Minimum GC content (default: 50).
        gc_max (int): Maximum GC content (default: 65).
        num_probes (int): Number of probes to select per gene (default: 10).
        iupac_mismatches (position:base): 
            List of positions (1-based) and IUPAC codes to introduce mismatches.        
            Recommended positions for mismatches are **15 and 16**.
            Example: 15:R,16:G
            **Note:** The total number of mismatches must be ≤2. Exceeding this limit may lead to unexpected behavior.

        max_errors (int): Maximum mismatches allowed during specificity checking.
        check_specificity (bool): Whether to check probe specificity against reference.

    Returns:
        DataFrame: DataFrame with extracted probe sequences (and specificity results if enabled).
    """

    targets = []

    seq_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    selected_features = pd.read_csv(selected_features, sep='\t')
    selected_features.index = selected_features['region']
    coverage_dict = dict(zip(selected_features['region'], selected_features['coverage']))

    if plp_length % 2 != 0:
        raise InputValueError("The size of the probe should be an even number", field="plp_length", code="odd_value_for_plp_length_provided")


    for keys in seq_dict:
        gene, region = keys.split("|")

        chr_name, start_end = region.split(":")
        initial_start = int(start_end.split("-")[0])
        seq = seq_dict[keys].seq
        seq_len = len(seq) - (plp_length - 1)

        coverage_value = coverage_dict.get(region, 0)
        if coverage_value < min_coverage:
            continue 

        for i in range(0, seq_len):
            tmp_seq = seq[i:i + plp_length]
            gc_content = (tmp_seq.count("G") + tmp_seq.count("C")) / len(tmp_seq) * 100

            if not (gc_min <= gc_content <= gc_max):
                continue
            if not any(nucleotide * 3 in tmp_seq for nucleotide in "ACGT"):
                continue

            start = initial_start + i
            end = start + plp_length - 1

            targets.append({
                "Probe_id": f"{gene}|{start}-{end}",
                "Gene": gene,
                "Region": f"{chr_name}:{start}-{end}",
                "Sequence": str(tmp_seq),
                "GC": gc_content,
                "Coverage": coverage_value,
                "Transcript_id": selected_features.loc[region, 'transcript_id']
            })

    targets_df = pd.DataFrame(targets)

    # Introduction of IUPAC mismatches
    if iupac_mismatches:
        targets_df = evaluate_ligation_junction(targets_df, iupac_mismatches=iupac_mismatches, plp_length=plp_length)


    # Check probe specificity against reference genome if requested
    if check_specificity:
        specificity_results = find_probes_in_targets(targets_df, reference_fasta, max_errors, output_file=f"{output_file}_specificity.csv")

        # Identify probes that have **only one unique match**
        unique_queries = specificity_results.groupby('query_name').size().reset_index(name='counts')
        unique_queries = unique_queries[unique_queries['counts'] == 1]['query_name']

        # Select only probes with a unique match
        targets_df = targets_df[targets_df['Probe_id'].isin(unique_queries)]

        # Merge mismatches into targets_df
        targets_df = targets_df.merge(
            specificity_results[['query_name', 'mismatches']],
            left_on='Probe_id',
            right_on='query_name',
            how='left'
        ).drop(columns=['query_name'])  # Drop duplicate column

        print(f"✅ Specificity results saved to {output_file}_specificity.csv")

    return targets_df



def parse_specificity_results(specificity_results):
    """
    Parse the specificity results from the CSV file.

    Args:
        specificity_results (str): Path to the specificity results CSV file.

    Returns:
        DataFrame: DataFrame containing parsed specificity results.
    """
    specificity_results.group_by('Probe_id').size().reset_index(name='counts')
    return pd.read_csv(specificity_results)

def evaluate_ligation_junction(targets, iupac_mismatches=None, plp_length=30, num_probes=10):
    """
    Evaluates the ligation junction of a probe and introduces mismatches if needed.

    Args:
        probe_seq (str): The probe sequence.
        iupac_mismatches (position:base): 
            List of positions (1-based) and IUPAC codes to introduce mismatches.        
            Recommended positions for mismatches are **15 and 16**.
            Example: 15:R,16:G
            **Note:** The total number of mismatches must be ≤2. Exceeding this limit may lead to unexpected behavior.
            
        plp_length (int): Probe length (default: 30).

    Returns:
        tuple: (updated probe sequence, ligation junction category)
    """
    # Add Ligation junction column to the DataFrame
    if 'Ligation junction' not in targets.columns:
        targets['Ligation junction'] = 'non-preferred'


    # Extract the ligation junction (2 bases around the center of the probe)
    junction_position = int((plp_length / 2) - 1)
    new_rows = []
    for idx in targets.index:

        probe_seq = targets.loc[idx]['Sequence']
        ligation_junction = probe_seq[junction_position] + probe_seq[junction_position + 2]

        # Determine the ligation status
        ligation_status = ligation_junctions_dict.get(ligation_junction, "non-preferred")
        targets.loc[idx, 'Ligation junction'] = ligation_status

        # Apply IUPAC mismatches if provided
        if iupac_mismatches is not None:
            if isinstance(iupac_mismatches, str):  # Only parse if it's a string
                iupac_mismatches = parse_iupac_mismatches(iupac_mismatches)

            num_mismatches = len(iupac_mismatches)

            if num_mismatches <= 2: # Limit to 2 mismatches
                for r in range(1, num_mismatches + 1): 
                    for subset in itertools.combinations(range(num_mismatches), r): # Generate all possible combinations of mismatches
                        selected_mismatches = [iupac_mismatches[i] for i in subset]
                        replacement_options = [IUPAC_CODES[symbol] for pos, symbol in selected_mismatches]

                        # Generate all possible combinations of replacements
                        for replacement in itertools.product(*replacement_options):
                            new_probe_seq = list(probe_seq)
                            new_id_suffix = []
                            for (pos, iupac_symbol), new_base in zip(selected_mismatches, replacement):
                                new_probe_seq[pos] = new_base
                                new_id_suffix.append(f"{pos}_{iupac_symbol}_{new_base}")
                            new_probe_seq = "".join(new_probe_seq)
                            new_probe_id = f"{targets.loc[idx, 'Probe_id']}|{'_'.join(new_id_suffix)}"
                            # Revaluate the ligation junction
                            new_ligation_junction = new_probe_seq[junction_position] + new_probe_seq[junction_position + 2]
                            new_ligation_status = ligation_junctions_dict.get(new_ligation_junction, "non-preferred")
                            new_row = targets.loc[idx].copy()
                            new_row['Sequence'] = new_probe_seq
                            new_row['Ligation junction'] = new_ligation_status
                            new_row['Probe_id'] = new_probe_id
                            new_rows.append((new_probe_id, new_row))
            else:
                raise InputValueError("The number of mismatches should be less than or equal to 2", field="iupac_mismatches", code="mismatches_exceed_limit")
   
    # Append the new rows to the DataFrame
    new_rows_df = pd.DataFrame([row[1] for row in new_rows], index=[row[0] for row in new_rows])
    targets = pd.concat([targets, new_rows_df])
    return targets

def parse_iupac_mismatches(mismatch_str):
    """
    Parses a string of mismatches formatted as "pos:base,pos:base" into a list of tuples.
    
    Args:
        mismatch_str (str): Mismatch input string (e.g., "5:R,10:G")
    
    Returns:
        list: A list of (position, base) tuples, e.g., [(5, 'R'), (10, 'G')].
    """
    mismatches = []
    try:
        for pair in mismatch_str.split(","):
            pos, base = pair.split(":")
            pos = int(pos.strip())  # Convert position to integer
            base = base.strip().upper()  # Ensure base is uppercase
            mismatches.append((pos, base))

    except (ValueError, IndexError):
        raise InputValueError("Invalid format for --iupac_mismatches. Use 'pos:base,pos:base', e.g., '5:R,10:G'.", 
                              field="iupac_mismatches", code="invalid_mismatch_format")
    
    return mismatches


In [124]:

#targets_df = pd.read_csv('../targets.txt', sep='\t')
#targets_df.set_index('Probe_id', inplace=True)
specificity_results = pd.read_csv('../probes_check.csv', sep=',')
selected_features = '../extract_features_output.txt'
#print(selected_features.head())

targets_df = targets_df_check = find_targets(
    selected_features=selected_features,
    fasta_file='../extract_seqs_output.fa',
    reference_fasta="../data/tmp_mrna.fa",
    plp_length=30,
    min_coverage=1,
    gc_min=50, gc_max=60,
    max_errors=1, check_specificity=True
)




Primary_probes: 673
references: [<SequenceRecord(name='ENSMUST00000218823 gene=ENSMUSG00000056073 gene_name=Grik2 seq_id=10 type=mrna', sequence='TCCGCCGCCGTCACTTGCGCTCTCGCTGGCGGCTGCGTGGCCGAGGCTGGGAGCCCGGGACTTCCCGCTGAACCGCCTCCTGCCGCAGCTCTGAGAG...')>, <SequenceRecord(name='ENSMUST00000217673 gene=ENSMUSG00000056073 gene_name=Grik2 seq_id=10 type=mrna', sequence='TTTTGGCATCCCTTCCTAAATAAAACTGATGAAGAGTCAATAGTTGCATTTTGCCTTTCCTCCATTTGGGGTCTAAGGTAAGGGCTTCTTTCCTCAT...')>, <SequenceRecord(name='ENSMUST00000218598 gene=ENSMUSG00000056073 gene_name=Grik2 seq_id=10 type=mrna', sequence='ATGAAGATTATTTCCCCAGTTTTAAGTAATCTAGTCTTCAGTCGCTCCATTAAAGTCCTGCTCTGCTTGTTGTGGATCGGATATTCGCAAGGAACCA...')>, <SequenceRecord(name='ENSMUST00000219509 gene=ENSMUSG00000056073 gene_name=Grik2 seq_id=10 type=mrna', sequence='TGATATCTGGATGTATATTCTGCTGGCTTACTTGGGTGTCAGTTGTGTGCTCTTTGTCATAGCCAGGTTTAGTCCCTATGAGTGGTATAATCCACAC...')>, <SequenceRecord(name='ENSMUST00000218441 gene=ENSMUSG00000056073 gene_name=Grik2 seq_id=10 type

Testing probes for specificity:  15%|█▍        | 1176/8076 [00:06<00:38, 181.53 alignments/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nimra236/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_35010/348349134.py", line 7, in <module>
    targets_df = targets_df_check = find_targets(
  File "/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_35010/607242675.py", line 261, in find_targets
    specificity_results = find_probes_in_targets(targets_df, reference_fasta, max_errors, output_file=f"{output_file}_specificity.csv")
  File "/var/folders/fs/brdpvm0j2yx21pxmd4h77m2h0000gn/T/ipykernel_35010/607242675.py", line 117, in find_probes_in_targets
    adapter = BackAdapter(probe_seq, max_errors=max_errors, min_overlap=len(probe_seq), indels=False)
  File "/Users/nimra236/anaconda3/lib/python3.10/site-packages/cutadapt/adapters.py", line 799, in __init__
    super().__init__(*args, **kwargs)
  File "/U